# Google Colab

In [8]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [28]:
import os
os.chdir('/gdrive/My Drive/Huawei')

In [29]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import datetime

In [30]:
root = '/gdrive/My Drive/Huawei/'
veriler = pd.read_csv(root + 'film_data.csv')
veriler2 = pd.read_csv(root + 'out.csv')



# VERİ ÖNİŞLEME

## Genre

In [14]:
veriler["genre"] = veriler["genre"].str.replace(" ", "")
veriler["genre"] = veriler["genre"].str.split(",")

mlb = MultiLabelBinarizer()
genre = veriler['genre']

encoded = pd.DataFrame(mlb.fit_transform(genre), columns=mlb.classes_, index=veriler.index)

## Country

In [15]:
veriler["country"] = veriler["country"].str.replace(" ", "")
veriler["country"] = veriler["country"].str.split(",")

mlb_country = MultiLabelBinarizer()
country = veriler['country']

encoded2 = pd.DataFrame(mlb_country.fit_transform(country), columns=mlb_country.classes_, index=veriler.index)


## Language

In [16]:
veriler["language"] = veriler["language"].str.replace(" ", "")
veriler["language"] = veriler["language"].str.split(",")

mlb_language = MultiLabelBinarizer()
language = veriler['language']

encoded3 = pd.DataFrame(mlb_language.fit_transform(language), columns=mlb_language.classes_, index=veriler.index)


## Votes

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

votes = sc.fit_transform(veriler[["votes"]])
votes = pd.DataFrame(votes, columns=["votes"], index=veriler.index)


## Duration

In [18]:
duration = veriler["duration"]

## Date Published

In [19]:
x = datetime.datetime.now()
date = veriler["date_published"]
date_list = []
for i in date:
    y = i.split("-")
    calculate_year = (x.year - int(y[0])) * 365
    calculate_month = (x.month - int(y[1])) * 30
    calculate_day = x.day - int(y[2])
    new_date = calculate_year + calculate_month + calculate_day
    date_list.append(new_date)

date_df = pd.DataFrame(date_list)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
date_df_new = sc.fit_transform(date_df)
date_df_new = pd.DataFrame(date_df_new, columns=["date_release"], index=veriler.index)


##Description

NLP.ipynb dosyasında işlenmiş olan topikler csv formatında buraya aktarılmıştır.

Topiklerin başlıkları 0'dan 9'a olacak şekilde numaralandırılmıştır.

In [20]:
from sklearn import preprocessing
veriler = pd.concat([veriler, veriler2], axis=1)
description = veriler.iloc[:, -1:]

ohe = preprocessing.OneHotEncoder()
description = ohe.fit_transform(description).toarray() # OHE işlemi

##Veri Birleştirme

In [21]:
feature1 = pd.DataFrame(data=encoded)
feature2 = pd.DataFrame(data=encoded2)
feature3 = pd.DataFrame(data=encoded3)
feature4 = pd.DataFrame(data=votes)
feature5 = pd.DataFrame(data=duration)
feature6 = pd.DataFrame(data=date_df_new)
feature7 = pd.DataFrame(data=description)

features = pd.concat([feature1, feature2], axis=1)
features = pd.concat([features, feature3], axis=1)
features = pd.concat([features, feature4], axis=1)
features = pd.concat([features, feature5], axis=1)
features = pd.concat([features, feature6], axis=1)
features = pd.concat([features, feature7], axis=1)

dependent_variable = veriler["avg_vote"]
dependent_df = pd.DataFrame(data=dependent_variable)

## BackWard Elimination yöntemi ile sm.OLS deki p değerine bakılarak p değeri yüksek olan özellikler elenmiştir. Elenen özellikler Country ve Language

In [22]:
import statsmodels.api as sm
X_l = features.iloc[:, :].values
X_l = np.array(X_l, dtype = float)
model = sm.OLS(dependent_df, X_l).fit() # P değeri yüksek olan language ve country kategorileri çıkartıldı.
#print(model.summary())

features_new = pd.concat([feature1, feature4], axis=1)
features_new = pd.concat([features_new, feature5], axis=1)
features_new = pd.concat([features_new, feature6], axis=1)
features_new = pd.concat([features_new, feature7], axis=1)


X_l = features_new.iloc[:, :].values
X_l = np.array(X_l, dtype=float)
model = sm.OLS(dependent_df, X_l).fit()
#print(model.summary())

print(features_new.columns)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Index([      'Action',        'Adult',    'Adventure',    'Animation',
          'Biography',       'Comedy',        'Crime',  'Documentary',
              'Drama',       'Family',      'Fantasy',    'Film-Noir',
            'History',       'Horror',        'Music',      'Musical',
            'Mystery',         'News',   'Reality-TV',      'Romance',
             'Sci-Fi',        'Sport',     'Thriller',          'War',
            'Western',        'votes',     'duration', 'date_release',
                    0,              1,              2,              3,
                    4,              5,              6,              7,
                    8],
      dtype='object')


#Train Test Verileri Bölme

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_new, dependent_df, test_size=0.33, random_state=0)


#Modelin Parametrelerinin Seçilmesi

In [ ]:
import xgboost as xg
from sklearn.model_selection import GridSearchCV

xgb_r = xg.XGBRegressor()

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb_r,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(x_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed:  5.9min finished


0.4106059715786025
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


#Modelin Eğitilmesi

In [24]:
import xgboost as xg
xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators = 500, seed = 123, nthread=4, learning_rate=.03, max_depth=6, colsample_bytree=0.7, min_child_weight=4, silent=1, subsample=0.7)

xgb_r.fit(x_train.values, y_train.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.03, max_delta_step=0,
             max_depth=6, min_child_weight=4, missing=None, n_estimators=500,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=123, silent=1,
             subsample=0.7, verbosity=1)

# Model Değerlendirme

In [25]:
from sklearn.metrics import r2_score
pred = xgb_r.predict(x_test.values)
print(r2_score(y_test.values, pred))

0.4266313323794315


# Model Kaydedilmesi

In [26]:
import pickle 

pickle.dump(xgb_r,open("model.kayit","wb"))

#yuklenen = pickle.load(open("model.kayit","rb")) # Kaydedilmiş modeli koda yüklüyoruz.